In [ ]:

import google.generativeai as genai
import os
import re
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
GOOGLE_API_KEY= ""
genai.configure(api_key=GOOGLE_API_KEY)

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model = "gemini-pro",api_key = GOOGLE_API_KEY)

In [5]:
model = genai.GenerativeModel(model_name = "gemini-pro")
model

genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)

In [6]:
llm_resto = ChatGoogleGenerativeAI(model = "gemini-pro",api_key = GOOGLE_API_KEY)

prompt_template_resto = PromptTemplate(
    input_variables=['age', 'gender', 'weight', 'height', 'veg_or_nonveg', 'disease', 'region', 'allergics', 'foodtype'],
    template="Diet Recommendation System:\n"
             "I want you to recommend 6 restaurants names, 6 breakfast names, 5 dinner names, and 6 workout names, "
             "based on the following criteria:\n"
             "Person age: {age}\n"
             "Person gender: {gender}\n"
             "Person weight: {weight}\n"
             "Person height: {height}\n"
             "Person veg_or_nonveg: {veg_or_nonveg}\n"
             "Person generic disease: {disease}\n"
             "Person region: {region}\n"
             "Person allergics: {allergics}\n"
             "Person foodtype: {foodtype}."
)

In [ ]:
# Updated prompt template with a stricter format
prompt_template_resto = PromptTemplate(
    input_variables=['age', 'gender', 'weight', 'height', 'veg_or_nonveg', 'disease', 'region', 'allergics', 'foodtype'],
    template="Based on the following personal information, please generate recommendations:\n"
             "- Age: {age}\n"
             "- Gender: {gender}\n"
             "- Weight: {weight} kg\n"
             "- Height: {height} feet\n"
             "- Diet preference (veg/non-veg): {veg_or_nonveg}\n"
             "- Health condition: {disease}\n"
             "- Region: {region}\n"
             "- Allergies: {allergics}\n"
             "- Preferred food type: {foodtype}\n\n"
             "Please provide:\n"
             "1. Six restaurant recommendations suitable for this individual.\n"
             "2. Six breakfast options aligned with the individual's preferences and health needs.\n"
             "3. Five dinner suggestions suitable for their diet and health conditions.\n"
             "4. Six workout suggestions appropriate for their health and fitness level.\n\n"
             "Format your response exactly as follows:\n\n"
             "Restaurants:\n- Restaurant 1\n- Restaurant 2\n...\n\n"
             "Breakfast:\n- Breakfast 1\n- Breakfast 2\n...\n\n"
             "Dinner:\n- Dinner 1\n- Dinner 2\n...\n\n"
             "Workouts:\n- Workout 1\n- Workout 2\n..."
)



chain_resto = prompt_template_resto | llm_resto


input_data = {
    'age': 60,
    'gender': 'male',
    'weight': 120,
    'height': 5,
    'veg_or_nonveg': 'veg',
    'disease': 'aneamia',
    'region': 'Pakistan',
    'allergics': 'Latex Allergy',
    'foodtype': 'Fruits'
}


results = chain_resto.invoke(input_data)


results_str = str(results)

In [20]:
results_str

"content='**Restaurants:**\\n\\n- **Mehfil-e-Azeem (Lahore):** Offers a wide variety of vegetarian dishes, including fruits and vegetable platters.\\n- **Green Corner (Karachi):** Specializes in healthy vegan and vegetarian cuisine, with a focus on organic and locally sourced ingredients.\\n- **Chatkhara (Islamabad):** Serves traditional Pakistani vegetarian dishes, including lentil soups, vegetable curries, and fresh salads.\\n- **Fruitilicious (Lahore):** A fruit-themed restaurant offering a range of fruit-based dishes, smoothies, and desserts.\\n- **Lasani (Karachi):** Provides a buffet-style dining experience with a large selection of vegetarian options, including salads, curries, and grilled vegetables.\\n- **Bhukhara (Islamabad):** Known for its extensive vegetarian menu with options like paneer tikka, vegetable biryani, and dal makhani.\\n\\n**Breakfast:**\\n\\n- **Oatmeal with fruit and nuts:** Rich in fiber, antioxidants, and iron, which is beneficial for anemia.\\n- **Banana 

In [ ]:
cleaned_results = re.sub(r'\s*\n\s*', '\n', results_str).strip()

print(cleaned_results)


content='**Restaurants:**\n\n1. **Humaira Hotel:** Serves a variety of vegetarian Pakistani dishes, including traditional favorites like chana masala and daal.\n2. **Bundoo Khan:** Known for its grilled vegetarian options, such as paneer tikka and vegetable seekh kebabs.\n3. **Lahore Chatkhara:** Offers a range of vegetarian chaat items, including pani puri and bhelpuri.\n4. **The Green Kitchen:** Specializes in healthy and nutritious vegetarian cuisine, with a focus on salads, wraps, and smoothies.\n5. **Cafe Zouk:** A cozy cafe that serves a selection of vegetarian sandwiches, salads, and soups.\n6. **Tuscany Court:** A fine-dining restaurant with a vegetarian menu featuring Italian and Mediterranean dishes.\n\n**Breakfast:**\n\n1. **Oatmeal with berries and nuts:** Whole-grain oats provide fiber and energy, while berries add antioxidants and vitamin C. Nuts contribute healthy fats and protein.\n2. **Fruit smoothie with spinach:** A refreshing and nutrient-rich blend of fruits, spina

In [ ]:
content_start = results_str.find('**Restaurants:**')
content_end = results_str.find('additional_kwargs')



**Restaurants:**\n\n1. **Humaira Hotel:** Serves a variety of vegetarian Pakistani dishes, including traditional favorites like chana masala and daal.\n2. **Bundoo Khan:** Known for its grilled vegetarian options, such as paneer tikka and vegetable seekh kebabs.\n3. **Lahore Chatkhara:** Offers a range of vegetarian chaat items, including pani puri and bhelpuri.\n4. **The Green Kitchen:** Specializes in healthy and nutritious vegetarian cuisine, with a focus on salads, wraps, and smoothies.\n5. **Cafe Zouk:** A cozy cafe that serves a selection of vegetarian sandwiches, salads, and soups.\n6. **Tuscany Court:** A fine-dining restaurant with a vegetarian menu featuring Italian and Mediterranean dishes.\n\n**Breakfast:**\n\n1. **Oatmeal with berries and nuts:** Whole-grain oats provide fiber and energy, while berries add antioxidants and vitamin C. Nuts contribute healthy fats and protein.\n2. **Fruit smoothie with spinach:** A refreshing and nutrient-rich blend of fruits, spinach, and d

In [30]:

if content_start != -1 and content_end != -1:
    content = results_str[content_start:content_end].strip()  # Extract the content

    # Print the structured content
    print("Diet Recommendations:")
    print("=====================\n")
    print(content.replace('\\n', '\n'))  # Replace '\n' for better formatting

else:
    print("No content found.")

Diet Recommendations:

**Restaurants:**

- The Olives Restaurant (Lahore): Offers a variety of vegetarian dishes rich in iron, such as spinach and lentil soup, and grilled tofu with vegetables.
- Savour Foods (Islamabad): Serves a wide selection of healthy and nutritious vegetarian options, including iron-fortified salads and stir-fried vegetables.
- La Vie (Karachi): Provides a sophisticated dining experience with a focus on healthy and balanced meals, featuring vegetarian dishes with iron-rich ingredients.
- Avari Hotel (Rawalpindi): Offers a buffet with a dedicated vegetarian section, including dishes like iron-rich legumes and leafy greens.
- The Pantry (Lahore): Serves a variety of vegetarian dishes with an emphasis on wholesome and nutritious ingredients, such as iron-rich beans and lentils.
- Kolachi (Karachi): Offers a variety of seafood dishes that are rich in iron, such as grilled salmon and tuna.

**Breakfast:**

- Oatmeal with berries, nuts, and seeds: Provides a good sourc